In [34]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
#import RPi.GPIO as GPIO
#from pynput import keyboard
#import math
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import time
%matplotlib inline

# MOTOR_A_A1=5
# MOTOR_A_B1=6
# MOTOR_B_A1=20
# MOTOR_B_B1=21

# GPIO.setmode(GPIO.BCM)
# GPIO.setup(MOTOR_A_A1,GPIO.OUT)
# GPIO.setup(MOTOR_A_B1,GPIO.OUT)
# GPIO.setup(MOTOR_B_A1,GPIO.OUT)
# GPIO.setup(MOTOR_B_B1,GPIO.OUT)

# MOTOR_A_A1_PWM=GPIO.PWM(MOTOR_A_A1,1000)
# MOTOR_A_B1_PWM=GPIO.PWM(MOTOR_A_B1,1000)
# MOTOR_B_A1_PWM=GPIO.PWM(MOTOR_B_A1,1000)
# MOTOR_B_B1_PWM=GPIO.PWM(MOTOR_B_B1,1000)

# MOTOR_A_A1_PWM.start(0)
# MOTOR_A_B1_PWM.start(0)
# MOTOR_B_A1_PWM.start(0)
# MOTOR_B_B1_PWM.start(0)

# def set_motor_speed(pwm_a, pwm_b, speed):
#     if speed >= 0:
#         pwm_a.ChangeDutyCycle(speed)
#         pwm_b.ChangeDutyCycle(0)
#     else:
#         pwm_a.ChangeDutyCycle(0)
#         pwm_b.ChangeDutyCycle(-speed)

# def on_press(key):
#     if key.char == 'w':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,100)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,100)
#         print('input is w')
#     elif key.char == 's':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,-100)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,-100)
#         print('input is s')
#     elif key.char == 'a':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 100)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 50)
#         print('input is a')
#     elif key.char == 'd':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 50)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 100)
#         print('input is d') 

# def on_release(key):
#     # if key.char == 'q':
#     if key == keyboard.Key.esc:
#         MOTOR_A_A1_PWM.stop()
#         MOTOR_A_B1_PWM.stop()
#         MOTOR_B_A1_PWM.stop()
#         MOTOR_B_B1_PWM.stop()
#         GPIO.cleanup()
#         return False
#     else:
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 0)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 0)

#grayscale
def grayscale(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#blur
def gaussian_blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),0)
#canny
def canny(img,low_threshold,high_threshold):
    return cv2.Canny(img,low_threshold,high_threshold)
#roi
def region_of_interest(img,vertices):
    mask=np.zeros_like(img)

    if len(img.shape)>2:
        channel_count=img.shape[2]
        ignore_mask_color=(255,)*channel_count
    else:
        ignore_mask_color=255

    cv2.fillPoly(mask,vertices,ignore_mask_color)
    
    masked_image=cv2.bitwise_and(img,mask)
    return masked_image

# def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
#     for line in lines:
#         for x1, y1, x2, y2 in line:
#             slope = (y2 - y1) / (x2 - x1 + 1e-5)  # 기울기 계산, 분모에 작은 값 추가하여 나누기 오류 방지
#             if abs(slope) > 0.5:  # 기울기가 일정 값 이하인 경우만 그리도록 설정
#                 cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    left_lane_lines = []  # 왼쪽 차선을 저장할 리스트
    right_lane_lines = []  # 오른쪽 차선을 저장할 리스트
    slope_threshold = 0.5  # 기울기가 이 값 이상이어야 차선으로 인정

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1 + 1e-5)  # 기울기 계산, 분모에 작은 값 추가하여 나누기 오류 방지
            if abs(slope) > slope_threshold:
                if slope > 0:  # 기울기가 양수이면 오른쪽 차선으로 판단
                    right_lane_lines.append(line)
                else:  # 기울기가 음수이면 왼쪽 차선으로 판단
                    left_lane_lines.append(line)

    # 왼쪽 차선 그리기
    for line in left_lane_lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

    # 오른쪽 차선 그리기
    for line in right_lane_lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img,rho,theta,threshold,min_line_len,max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:  # Check if lines is None
        print("No lines found!")
        return np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img,initial_img,a=0.8,b=1.,c=0.):
    return cv2.addWeighted(initial_img,a,img,b,c)

# def find_vanishing_point(lines, imshape):
#     # 수평 차선들의 중점을 계산
#     horizontal_midpoints = []
#     for line in lines:
#         try:
#             for x1, y1, x2, y2 in line:
#                 horizontal_midpoints.append(((x1 + x2) / 2, (y1 + y2) / 2))
#         except:
#             horizontal_midpoints.append(((x1 + x2) / 2, (y1 + y2) / 2))

#     # 수평 차선 중점들을 기반으로 소실점 계산
#     if horizontal_midpoints:
#         vanishing_point = np.mean(horizontal_midpoints, axis=0, dtype=int)
#         vanishing_point = (vanishing_point[0], imshape[0] // 2)  # 소실점의 y 좌표를 이미지 중앙으로 조정
#         return vanishing_point
#     else:
#         return None

# def find_vanishing_point(lines, imshape):
#     # 수평 차선들의 중점을 계산
#     horizontal_midpoints = []
#     for line in lines:
#         if line is not None and len(line) == 1 and len(line[0]) == 4:  
#             x1, y1, x2, y2 = line[0]
#             horizontal_midpoints.append(((x1 + x2) / 2, (y1 + y2) / 2))

#     # 수평 차선 중점들을 기반으로 소실점 계산
#     if horizontal_midpoints:
#         vanishing_point = np.mean(horizontal_midpoints, axis=0, dtype=int)
#         vanishing_point = (vanishing_point[0], imshape[0] // 2)  # 소실점의 y 좌표를 이미지 중앙으로 조정
#         return vanishing_point
#     else:
#         return None

def find_vanishing_point(lines, imshape):
    left_lane_lines = []  # 왼쪽 차선을 저장할 리스트
    right_lane_lines = []  # 오른쪽 차선을 저장할 리스트
    slope_threshold = 0.5  # 기울기가 이 값 이상이어야 차선으로 인정

    for line in lines:
        if line is not None and len(line) == 1 and len(line[0]) == 4:
            x1, y1, x2, y2 = line[0]
            # 기울기 계산, 분모에 작은 값 추가하여 나누기 오류 방지
            slope = (y2 - y1) / ((x2 - x1) + 1e-5)
            if abs(slope) > slope_threshold:
                if slope > 0:  # 기울기가 양수이면 오른쪽 차선으로 판단
                    right_lane_lines.append(line)
                else:  # 기울기가 음수이면 왼쪽 차선으로 판단
                    left_lane_lines.append(line)

    # 왼쪽 차선의 평균 기울기와 평균 위치 계산
    left_slope = np.mean([(y2 - y1) / ((x2 - x1) + 1e-5) for line in left_lane_lines for x1, y1, x2, y2 in line])
    # 왼쪽 차선이 감지되지 않거나 기울기가 0이면 기본값 사용
    if np.isnan(left_slope) or left_slope == 0:
        left_slope = 0.001  # 0으로 나누는 것을 방지하기 위한 작은 값 사용
    left_line = [int(imshape[1] * 0.45), imshape[0], int((imshape[0] / 2) / left_slope) + imshape[1] // 2, int(imshape[0] / 2)]

    # 오른쪽 차선의 평균 기울기와 평균 위치 계산
    right_slope = np.mean([(y2 - y1) / ((x2 - x1) + 1e-5) for line in right_lane_lines for x1, y1, x2, y2 in line])
    # 오른쪽 차선이 감지되지 않으면 기본값 사용
    if np.isnan(right_slope):
        right_slope = 0.001
    right_line = [int(imshape[1] * 0.55), imshape[0], int((imshape[0] / 2) / right_slope) + imshape[1] // 2, int(imshape[0] / 2)]

    return left_line, right_line






kernel_size=5
low_threshold=50
high_threshold=200

rho=5
theta=np.pi/180
threshold=90
min_line_len=120
max_line_gap=150

# camera = cv2.VideoCapture(0)
# camera.set(3, 640)
# camera.set(4, 480)
# listener = keyboard.Listener(on_press=on_press, on_release=on_release)
# listener.start()
img = mpimg.imread('raspi_test.jpg')

while True:
    # ret, img = camera.read()
    # img = cv2.flip(img, 0)

    gray = grayscale(img)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, low_threshold, high_threshold)
    mask = np.zeros_like(img)

    imshape = img.shape
    vertices = np.array([[(0, imshape[0]), (100, imshape[0]/2), (500, imshape[0]/2), (imshape[1], imshape[0])]], dtype=np.int32)
    mask = region_of_interest(edges, vertices)

    lines = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)
    lines_edges = weighted_img(lines, img, a=0.8, b=1., c=0.)

    # 소실점 계산
    vanishing_point = find_vanishing_point(lines, imshape)

    # 소실점을 이미지에 그리기
    if vanishing_point:
        print(vanishing_point)
        cv2.circle(lines_edges, vanishing_point, 5, (0, 255, 0), -1)
    else:
        print('none')

    cv2.imshow('myimg', lines_edges)

    if cv2.waitKey(1) == ord('q'):
        #listener.join()
        break

#camera.release()
cv2.destroyAllWindows()

([288, 480, 240320, 240], [352, 480, 240320, 240])


error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type


In [59]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    left_lane_lines = []  
    right_lane_lines = []  
    slope_threshold = 0.5  

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1 + 1e-5)
            if abs(slope) > slope_threshold:
                if slope > 0:
                    right_lane_lines.append(line)
                else:
                    left_lane_lines.append(line)

    left_line, right_line = find_representative_lines(left_lane_lines, right_lane_lines, img.shape)

    # 왼쪽 차선 그리기
    #cv2.line(img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), color, thickness)

    # 오른쪽 차선 그리기
    #cv2.line(img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), color, thickness)

def find_representative_lines(left_lane_lines, right_lane_lines, imshape):
    left_slope = np.mean([(y2 - y1) / ((x2 - x1) + 1e-5) for line in left_lane_lines for x1, y1, x2, y2 in line])
    print(left_slope)
    if np.isnan(left_slope) or left_slope == 0:
        left_slope = 0.001
    left_line = [int(imshape[1] * 0.45), imshape[0], int((imshape[0] / 2) / left_slope) + imshape[1] // 2, int(imshape[0] / 2)]
    # for line in left_lane_lines:
    #     for x1,y1,x2,y2 in line:
    #         left_line = [x1,x2,y1,y2]

    right_slope = np.mean([(y2 - y1) / ((x2 - x1) + 1e-5) for line in right_lane_lines for x1, y1, x2, y2 in line])
    if np.isnan(right_slope):
        right_slope = 0.001
    right_line = [int(imshape[1] * 0.55), imshape[0], int((imshape[0] / 2) / right_slope) + imshape[1] // 2, int(imshape[0] / 2)]

    return left_line, right_line

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:
        print("No lines found!")
        return np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)

def find_vanishing_point(lines, imshape):
    left_lane_lines = []  
    right_lane_lines = []  
    slope_threshold = 0.5  

    for line in lines:
        if line is not None and len(line) == 1 and len(line[0]) == 4:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / ((x2 - x1) + 1e-5)
            if abs(slope) > slope_threshold:
                if slope > 0:
                    right_lane_lines.append(line)
                else:
                    left_lane_lines.append(line)

    left_line, right_line = find_representative_lines(left_lane_lines, right_lane_lines, imshape)
    vanishing_point = ((left_line[0] + right_line[0]) // 2, (left_line[1] + right_line[1]) // 2)

    return vanishing_point

kernel_size = 5
low_threshold = 50
high_threshold = 200

rho = 5
theta = np.pi/180
threshold = 90
min_line_len = 120
max_line_gap = 150

img = mpimg.imread('raspi_test.jpg')

gray = grayscale(img)
blur_gray = gaussian_blur(gray, kernel_size)
edges = canny(blur_gray, low_threshold, high_threshold)

imshape = img.shape
vertices = np.array([[(0, imshape[0]), (100, imshape[0]/2), (500, imshape[0]/2), (imshape[1], imshape[0])]], dtype=np.int32)
mask = region_of_interest(edges, vertices)

lines = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)
lines_edges = weighted_img(lines, img, a=0.8, b=1., c=0.)

vanishing_point = find_vanishing_point(lines, imshape)
if vanishing_point:
    print(vanishing_point)
    cv2.circle(lines_edges, vanishing_point, 5, (0, 255, 0), -1)
else:
    print('none')

cv2.imshow('myimg', lines_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


-1.1767730753494343
nan
(320, 480)


In [ ]:
import RPi.GPIO as GPIO
from pynput import keyboard
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

MOTOR_A_A1=5
MOTOR_A_B1=6
MOTOR_B_A1=20
MOTOR_B_B1=21

GPIO.setmode(GPIO.BCM)
GPIO.setup(MOTOR_A_A1,GPIO.OUT)
GPIO.setup(MOTOR_A_B1,GPIO.OUT)
GPIO.setup(MOTOR_B_A1,GPIO.OUT)
GPIO.setup(MOTOR_B_B1,GPIO.OUT)

MOTOR_A_A1_PWM=GPIO.PWM(MOTOR_A_A1,1000)
MOTOR_A_B1_PWM=GPIO.PWM(MOTOR_A_B1,1000)
MOTOR_B_A1_PWM=GPIO.PWM(MOTOR_B_A1,1000)
MOTOR_B_B1_PWM=GPIO.PWM(MOTOR_B_B1,1000)

MOTOR_A_A1_PWM.start(0)
MOTOR_A_B1_PWM.start(0)
MOTOR_B_A1_PWM.start(0)
MOTOR_B_B1_PWM.start(0)

def set_motor_speed(pwm_a, pwm_b, speed):
    if speed >= 0:
        pwm_a.ChangeDutyCycle(speed)
        pwm_b.ChangeDutyCycle(0)
    else:
        pwm_a.ChangeDutyCycle(0)
        pwm_b.ChangeDutyCycle(-speed)

def on_press(key):
    if key.char == 'w':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,100)
        print('input is w')
    elif key.char == 's':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,-100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,-100)
        print('input is s')
    elif key.char == 'a':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 50)
        print('input is a')
    elif key.char == 'd':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 50)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 100)
        print('input is d') 

def on_release(key):
    # if key.char == 'q':
    if key == keyboard.Key.esc:
        MOTOR_A_A1_PWM.stop()
        MOTOR_A_B1_PWM.stop()
        MOTOR_B_A1_PWM.stop()
        MOTOR_B_B1_PWM.stop()
        GPIO.cleanup()
        return False
    else:
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 0)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 0)

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    left_lane_lines = []  
    right_lane_lines = []  
    slope_threshold = 0.5  

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1 + 1e-5)
            if abs(slope) > slope_threshold:
                if slope > 0:
                    right_lane_lines.append(line)
                else:
                    left_lane_lines.append(line)

    left_line, right_line = find_representative_lines(left_lane_lines, right_lane_lines, img.shape)

    # 왼쪽 차선 그리기
    #cv2.line(img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), color, thickness)

    # 오른쪽 차선 그리기
    #cv2.line(img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), color, thickness)

def find_representative_lines(left_lane_lines, right_lane_lines, imshape):
    left_slope = np.mean([(y2 - y1) / ((x2 - x1) + 1e-5) for line in left_lane_lines for x1, y1, x2, y2 in line])
    print(left_slope)
    if np.isnan(left_slope) or left_slope == 0:
        left_slope = 0.001
    left_line = [int(imshape[1] * 0.45), imshape[0], int((imshape[0] / 2) / left_slope) + imshape[1] // 2, int(imshape[0] / 2)]

    right_slope = np.mean([(y2 - y1) / ((x2 - x1) + 1e-5) for line in right_lane_lines for x1, y1, x2, y2 in line])
    if np.isnan(right_slope):
        right_slope = 0.001
    right_line = [int(imshape[1] * 0.55), imshape[0], int((imshape[0] / 2) / right_slope) + imshape[1] // 2, int(imshape[0] / 2)]

    return left_line, right_line

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:
        print("No lines found!")
        return np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)

def find_vanishing_point(lines, imshape):
    left_lane_lines = []  
    right_lane_lines = []  
    slope_threshold = 0.5  

    for line in lines:
        if line is not None and len(line) == 1 and len(line[0]) == 4:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / ((x2 - x1) + 1e-5)
            if abs(slope) > slope_threshold:
                if slope > 0:
                    right_lane_lines.append(line)
                else:
                    left_lane_lines.append(line)

    left_line, right_line = find_representative_lines(left_lane_lines, right_lane_lines, imshape)
    vanishing_point = ((left_line[0] + right_line[0]) // 2, (left_line[1] + right_line[1]) // 2)

    return vanishing_point

kernel_size = 5
low_threshold = 50
high_threshold = 200

rho = 5
theta = np.pi/180
threshold = 90
min_line_len = 120
max_line_gap = 150

camera = cv2.VideoCapture(0)
camera.set(3, 640)
camera.set(4, 480)
listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()

#img = mpimg.imread('raspi_test.jpg')

while True:
    ret, img = camera.read()
    img = cv2.flip(img, 0)

    gray = grayscale(img)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, low_threshold, high_threshold)

    imshape = img.shape
    vertices = np.array([[(0, imshape[0]), (100, imshape[0]/2), (500, imshape[0]/2), (imshape[1], imshape[0])]], dtype=np.int32)
    mask = region_of_interest(edges, vertices)

    lines = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)
    lines_edges = weighted_img(lines, img, a=0.8, b=1., c=0.)

    vanishing_point = find_vanishing_point(lines, imshape)
    if vanishing_point:
        print(vanishing_point)
        cv2.circle(lines_edges, vanishing_point, 5, (0, 255, 0), -1)
    else:
        print('none')

    cv2.imshow('myimg', lines_edges)

    if cv2.waitKey(1) == ord('q'):
        listener.join()
        break

camera.release()
cv2.destroyAllWindows()

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import numpy as np
import cv2
import RPi.GPIO as GPIO
import time
from pynput import keyboard
#import math
# %matplotlib inline

MOTOR_A_A1=5
MOTOR_A_B1=6
MOTOR_B_A1=20
MOTOR_B_B1=21

GPIO.setmode(GPIO.BCM)
GPIO.setup(MOTOR_A_A1,GPIO.OUT)
GPIO.setup(MOTOR_A_B1,GPIO.OUT)
GPIO.setup(MOTOR_B_A1,GPIO.OUT)
GPIO.setup(MOTOR_B_B1,GPIO.OUT)

MOTOR_A_A1_PWM=GPIO.PWM(MOTOR_A_A1,1000)
MOTOR_A_B1_PWM=GPIO.PWM(MOTOR_A_B1,1000)
MOTOR_B_A1_PWM=GPIO.PWM(MOTOR_B_A1,1000)
MOTOR_B_B1_PWM=GPIO.PWM(MOTOR_B_B1,1000)

MOTOR_A_A1_PWM.start(0)
MOTOR_A_B1_PWM.start(0)
MOTOR_B_A1_PWM.start(0)
MOTOR_B_B1_PWM.start(0)

def set_motor_speed(pwm_a, pwm_b, speed):
    if speed >= 0:
        pwm_a.ChangeDutyCycle(speed)
        pwm_b.ChangeDutyCycle(0)
    else:
        pwm_a.ChangeDutyCycle(0)
        pwm_b.ChangeDutyCycle(-speed)

def on_press(key):
    if key.char == 'w':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,100)
        print('input is w')
    elif key.char == 's':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,-100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,-100)
        print('input is s')
    elif key.char == 'a':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 50)
        print('input is a')
    elif key.char == 'd':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 50)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 100)
        print('input is d') 

def on_release(key):
    # if key.char == 'q':
    if key == keyboard.Key.esc:
        MOTOR_A_A1_PWM.stop()
        MOTOR_A_B1_PWM.stop()
        MOTOR_B_A1_PWM.stop()
        MOTOR_B_B1_PWM.stop()
        GPIO.cleanup()
        return False
    else:
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 0)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 0)

#grayscale
def grayscale(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#blur
def gaussian_blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),0)
#canny
def canny(img,low_threshold,high_threshold):
    return cv2.Canny(img,low_threshold,high_threshold)
#roi
def region_of_interest(img,vertices):
    mask=np.zeros_like(img)

    if len(img.shape)>2:
        channel_count=img.shape[2]
        ignore_mask_color=(255,)*channel_count
    else:
        ignore_mask_color=255

    cv2.fillPoly(mask,vertices,ignore_mask_color)
    
    masked_image=cv2.bitwise_and(img,mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1 + 1e-5)  # 기울기 계산, 분모에 작은 값 추가하여 나누기 오류 방지
            if abs(slope) > 0.5:  # 기울기가 일정 값 이하인 경우만 그리도록 설정
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    if lines is None or len(lines) < 2:
        print("No lines found!")
        return np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img,initial_img,a=0.8,b=1.,c=0.):
    return cv2.addWeighted(initial_img,a,img,b,c)

def find_intersection(left_lane, right_lane):
    # left_lane와 right_lane은 각각 (x1, y1, x2, y2) 형태의 좌표를 가진 리스트입니다.
    # 좌표에서 중점을 찾아 교점을 계산합니다.
    left_midpoint = ((left_lane[0] + left_lane[2]) // 2, (left_lane[1] + left_lane[3]) // 2)
    right_midpoint = ((right_lane[0] + right_lane[2]) // 2, (right_lane[1] + right_lane[3]) // 2)

    # 중점을 이용하여 교점을 계산합니다.
    intersection_x = (left_midpoint[0] + right_midpoint[0]) // 2
    intersection_y = (left_midpoint[1] + right_midpoint[1]) // 2

    return intersection_x, intersection_y


kernel_size=5
low_threshold=50
high_threshold=200

rho=5
theta=np.pi/180
threshold=90
min_line_len=120
max_line_gap=150

camera = cv2.VideoCapture(0)
camera.set(3, 640)
camera.set(4, 480)
listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()

while True:
    ret, img = camera.read()
    img = cv2.flip(img, 0)

    gray = grayscale(img)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, low_threshold, high_threshold)
    mask = np.zeros_like(img)

    imshape = img.shape
    vertices = np.array([[(0, imshape[0]), (100, imshape[0]/2), (500, imshape[0]/2), (imshape[1], imshape[0])]], dtype=np.int32)
    mask = region_of_interest(edges, vertices)

    lines = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)

    if np.sum(lines) == 0:
        print("No lines found!")
        continue

    lines_edges = weighted_img(lines, img, a=0.8, b=1., c=0.)

    # 선의 좌표를 가져와서 교점을 찾습니다.
    left_lane = lines[0][0]  # 왼쪽 차선 좌표
    right_lane = lines[1][0]  # 오른쪽 차선 좌표

    # 교점을 찾습니다.
    intersection_x, intersection_y = find_intersection(left_lane, right_lane)

    # 교점을 표시합니다.
    cv2.circle(lines_edges, (intersection_x, intersection_y), 5, (0, 255, 0), -1)

    cv2.imshow('myimg', lines_edges)

    if cv2.waitKey(1) == ord('q'):
        listener.join()
        break

camera.release()
cv2.destroyAllWindows()

In [73]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import numpy as np
import cv2
# import RPi.GPIO as GPIO
# from pynput import keyboard
import time
#import math
# %matplotlib inline

# MOTOR_A_A1=5
# MOTOR_A_B1=6
# MOTOR_B_A1=20
# MOTOR_B_B1=21

# GPIO.setmode(GPIO.BCM)
# GPIO.setup(MOTOR_A_A1,GPIO.OUT)
# GPIO.setup(MOTOR_A_B1,GPIO.OUT)
# GPIO.setup(MOTOR_B_A1,GPIO.OUT)
# GPIO.setup(MOTOR_B_B1,GPIO.OUT)

# MOTOR_A_A1_PWM=GPIO.PWM(MOTOR_A_A1,1000)
# MOTOR_A_B1_PWM=GPIO.PWM(MOTOR_A_B1,1000)
# MOTOR_B_A1_PWM=GPIO.PWM(MOTOR_B_A1,1000)
# MOTOR_B_B1_PWM=GPIO.PWM(MOTOR_B_B1,1000)

# MOTOR_A_A1_PWM.start(0)
# MOTOR_A_B1_PWM.start(0)
# MOTOR_B_A1_PWM.start(0)
# MOTOR_B_B1_PWM.start(0)

# def set_motor_speed(pwm_a, pwm_b, speed):
#     if speed >= 0:
#         pwm_a.ChangeDutyCycle(speed)
#         pwm_b.ChangeDutyCycle(0)
#     else:
#         pwm_a.ChangeDutyCycle(0)
#         pwm_b.ChangeDutyCycle(-speed)

# def on_press(key):
#     if key.char == 'w':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,100)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,100)
#         print('input is w')
#     elif key.char == 's':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,-100)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,-100)
#         print('input is s')
#     elif key.char == 'a':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 100)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 50)
#         print('input is a')
#     elif key.char == 'd':
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 50)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 100)
#         print('input is d') 

# def on_release(key):
#     # if key.char == 'q':
#     if key == keyboard.Key.esc:
#         MOTOR_A_A1_PWM.stop()
#         MOTOR_A_B1_PWM.stop()
#         MOTOR_B_A1_PWM.stop()
#         MOTOR_B_B1_PWM.stop()
#         GPIO.cleanup()
#         return False
#     else:
#         set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 0)
#         set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 0)

#grayscale
def grayscale(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#blur
def gaussian_blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),0)
#canny
def canny(img,low_threshold,high_threshold):
    return cv2.Canny(img,low_threshold,high_threshold)
#roi
def region_of_interest(img,vertices):
    mask=np.zeros_like(img)

    if len(img.shape)>2:
        channel_count=img.shape[2]
        ignore_mask_color=(255,)*channel_count
    else:
        ignore_mask_color=255

    cv2.fillPoly(mask,vertices,ignore_mask_color)
    
    masked_image=cv2.bitwise_and(img,mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            #slope = (y2 - y1) / (x2 - x1 + 1e-5)  # 기울기 계산, 분모에 작은 값 추가하여 나누기 오류 방지
            #if abs(slope) > 0.5:  # 기울기가 일정 값 이하인 경우만 그리도록 설정
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def draw_fit_line(img, lines, color=[255, 0, 0], thickness=10): # 대표선 그리기
        cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness)

def get_fitline(img, f_lines): # 대표선 구하기   
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0]*2,2)
    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx + x) , int(img.shape[0]/2+100)
    
    result = [x1,y1,x2,y2]
    return result

def hough_lines(img,rho,theta,threshold,min_line_len,max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:  # Check if lines is None
        print("No lines found!")
        return np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img,initial_img,a=0.8,b=1.,c=0.):
    return cv2.addWeighted(initial_img,a,img,b,c)

kernel_size=5
low_threshold=50
high_threshold=200

rho=5
theta=np.pi/180
threshold=90
min_line_len=120
max_line_gap=150

# camera = cv2.VideoCapture(0)
# camera.set(3, 640)
# camera.set(4, 480)
# listener = keyboard.Listener(on_press=on_press, on_release=on_release)
# listener.start()

img = mpimg.imread('raspi_test.jpg')

while True:
    # ret, img = camera.read()
    # img = cv2.flip(img, 0)

    gray = grayscale(img)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, low_threshold, high_threshold)
    mask = np.zeros_like(img)

    imshape = img.shape
    vertices = np.array([[(0, imshape[0]), (100, imshape[0]/2), (500, imshape[0]/2), (imshape[1], imshape[0])]], dtype=np.int32)
    mask = region_of_interest(edges, vertices)

    line_arr = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)
    #lines_edges = weighted_img(lines, img, a=0.8, b=1., c=0.)
    line_arr = np.squeeze(line_arr)
    print(line_arr)
        
    # 기울기 구하기
    slope_degree = (np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:,2]) * 180) / np.pi

    # 수평 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)<160]
    slope_degree = slope_degree[np.abs(slope_degree)<160]
    # 수직 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]

    # 필터링된 직선 버리기
    L_lines, R_lines = line_arr[(slope_degree>0),:], line_arr[(slope_degree<0),:]
    temp = np.zeros((imshape[0], imshape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]
    # 왼쪽, 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(img,L_lines)
    right_fit_line = get_fitline(img,R_lines)
    # 대표선 그리기
    draw_fit_line(temp, left_fit_line)
    draw_fit_line(temp, right_fit_line)

    result = weighted_img(temp, img) # 원본 이미지에 검출된 선 overlap

    cv2.imshow('myimg', result)

    if cv2.waitKey(1) == ord('q'):
        #listener.join()
        break

#camera.release()
cv2.destroyAllWindows()

[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [255   0   0]
  [255   0   0]
  [255   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [255   0   0]
  [255   0   0]
  [255   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [255   0   0]
  [255   0   0]
  [255   0   0]]]


IndexError: boolean index did not match indexed array along dimension 1; dimension is 640 but corresponding boolean dimension is 3

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import numpy as np
import cv2
import RPi.GPIO as GPIO
import time
from pynput import keyboard
#import math
# %matplotlib inline

MOTOR_A_A1=5
MOTOR_A_B1=6
MOTOR_B_A1=20
MOTOR_B_B1=21

GPIO.setmode(GPIO.BCM)
GPIO.setup(MOTOR_A_A1,GPIO.OUT)
GPIO.setup(MOTOR_A_B1,GPIO.OUT)
GPIO.setup(MOTOR_B_A1,GPIO.OUT)
GPIO.setup(MOTOR_B_B1,GPIO.OUT)

MOTOR_A_A1_PWM=GPIO.PWM(MOTOR_A_A1,1000)
MOTOR_A_B1_PWM=GPIO.PWM(MOTOR_A_B1,1000)
MOTOR_B_A1_PWM=GPIO.PWM(MOTOR_B_A1,1000)
MOTOR_B_B1_PWM=GPIO.PWM(MOTOR_B_B1,1000)

MOTOR_A_A1_PWM.start(0)
MOTOR_A_B1_PWM.start(0)
MOTOR_B_A1_PWM.start(0)
MOTOR_B_B1_PWM.start(0)

def set_motor_speed(pwm_a, pwm_b, speed):
    if speed >= 0:
        pwm_a.ChangeDutyCycle(speed)
        pwm_b.ChangeDutyCycle(0)
    else:
        pwm_a.ChangeDutyCycle(0)
        pwm_b.ChangeDutyCycle(-speed)

def on_press(key):
    if key.char == 'w':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,100)
        print('input is w')
    elif key.char == 's':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM,-100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM,-100)
        print('input is s')
    elif key.char == 'a':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 100)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 50)
        print('input is a')
    elif key.char == 'd':
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 50)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 100)
        print('input is d') 

def on_release(key):
    # if key.char == 'q':
    if key == keyboard.Key.esc:
        MOTOR_A_A1_PWM.stop()
        MOTOR_A_B1_PWM.stop()
        MOTOR_B_A1_PWM.stop()
        MOTOR_B_B1_PWM.stop()
        GPIO.cleanup()
        return False
    else:
        set_motor_speed(MOTOR_A_A1_PWM, MOTOR_A_B1_PWM, 0)
        set_motor_speed(MOTOR_B_A1_PWM, MOTOR_B_B1_PWM, 0)

#grayscale
def grayscale(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#blur
def gaussian_blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),0)
#canny
def canny(img,low_threshold,high_threshold):
    return cv2.Canny(img,low_threshold,high_threshold)
#roi
def region_of_interest(img,vertices):
    mask=np.zeros_like(img)

    if len(img.shape)>2:
        channel_count=img.shape[2]
        ignore_mask_color=(255,)*channel_count
    else:
        ignore_mask_color=255

    cv2.fillPoly(mask,vertices,ignore_mask_color)
    
    masked_image=cv2.bitwise_and(img,mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1 + 1e-5)  # 기울기 계산, 분모에 작은 값 추가하여 나누기 오류 방지
            if abs(slope) > 0.5:  # 기울기가 일정 값 이하인 경우만 그리도록 설정
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img,rho,theta,threshold,min_line_len,max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:  # Check if lines is None
        print("No lines found!")
        return np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img,initial_img,a=0.8,b=1.,c=0.):
    return cv2.addWeighted(initial_img,a,img,b,c)

kernel_size=5
low_threshold=50
high_threshold=200

rho=5
theta=np.pi/180
threshold=90
min_line_len=120
max_line_gap=150

camera = cv2.VideoCapture(0)
camera.set(3, 640)
camera.set(4, 480)
listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()

while True:
    ret, img = camera.read()
    img = cv2.flip(img, 0)

    gray = grayscale(img)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, low_threshold, high_threshold)
    mask = np.zeros_like(img)

    imshape = img.shape
    vertices = np.array([[(0, imshape[0]), (100, imshape[0]/2), (500, imshape[0]/2), (imshape[1], imshape[0])]], dtype=np.int32)
    mask = region_of_interest(edges, vertices)

    lines = hough_lines(mask, rho, theta, threshold, min_line_len, max_line_gap)
    lines_edges = weighted_img(lines, img, a=0.8, b=1., c=0.)

    cv2.imshow('myimg', lines_edges)

    if cv2.waitKey(1) == ord('q'):
        listener.join()
        break

camera.release()
cv2.destroyAllWindows()